# ChatGPTをAPIから利用するために

## 3-3 入出力の長さの制限や課金に影響する「トークン」

### Tokenizerとtiktokenの紹介

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

### 日本語のトークン数について

In [ ]:
text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

## 3-4 Chat Completions APIにふれる環境準備

### Google Colabのノートブック作成

In [ ]:
print("Hello World")

### OpenAIのAPIキーの準備

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

## 3-5 Chat Completions APIをさわってみる

### OpenAIのライブラリ

In [ ]:
!pip install openai

### Chat Completions APIの呼び出し

In [ ]:
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(response)

### 会話履歴を踏まえた応答を得る

In [ ]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

### ストリーミングで応答を得る

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."}
  ],
  stream=True
)

for chunk in response:
  choice = chunk["choices"][0]
  if choice["finish_reason"] is None:
    print(choice["delta"]["content"])

### （コラム）Completions API

In [ ]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Hello! I'm John."
)

print(response)

In [ ]:
prompt = """Human: Hello! I'm John.
AI: Nice to meet you, John!
Human: Do you know my name?
AI: """

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt
)
print(response)

## 3-6 Function calling

### Function callingのサンプルコード

[OpenAI の公式ドキュメント](https://platform.openai.com/docs/guides/gpt/function-calling) をもとに一部改変したコードです。

In [ ]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Tokyo",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [{"role": "user", "content": "What's the weather like in Tokyo?"}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(response)

In [ ]:
response_message = response["choices"][0]["message"]

available_functions = {
    "get_current_weather": get_current_weather,
}
function_name = response_message["function_call"]["name"]
fuction_to_call = available_functions[function_name]
function_args = json.loads(response_message["function_call"]["arguments"])

function_response = fuction_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)

print(function_response)

In [ ]:
messages.append(response_message)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)

In [ ]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

print(second_response)